In [1]:
import xarray as xr 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from joblib import dump, load

for wind

In [2]:
# Define the locations and their coordinates
locations = {
    "Bern": (2595842.15, 1201548.69),
    "Brienz": (2646563.43, 1177603.88),
    "Buochs": (2674514.65, 1203568.08),
    "Interlaken": (2632995.62, 1171399.67),
    "Lugano": (2713926.15, 1094747.56),
    "Salavaux": (2569175.20, 1195846.24),
    "Sempach": (2657011.60, 1219718.50),
    "Gordevio": (2700590.96, 1119701.60),
    "Thun": (2614554.68, 1178201.33),
    "Morges": (2527116.10, 1150785.39)
}

# Open your datasets
path = '/scratch2/cwenger/ICON-CH1_monthly/ICON-CH1_gust_data/ICON-CH1-EPS_2024-08-12T12_agg.nc'
ds = xr.open_dataset(path)  # Replace with your actual dataset

# Loop through each location to find the nearest x and y and save variables
for name, (loc_x, loc_y) in locations.items():
    # Find the nearest grid cell
    nearest_x = ds.sel(x=loc_x, y=loc_y, method="nearest").x.values
    nearest_y = ds.sel(x=loc_x, y=loc_y, method="nearest").y.values

    # Save variables dynamically
    globals()[f"{name}_cell_x"] = nearest_x
    globals()[f"{name}_cell_y"] = nearest_y


locations_df = pd.DataFrame([(name, coord[0], coord[1]) for name, coord in locations.items()], columns=['name', 'x_coordinate', 'y_coordinate'])
# Add x_cell and y_cell columns to the dataframe
locations_df['x_cell_nearest'] = locations_df['name'].apply(lambda name: globals().get(f"{name}_cell_x"))
locations_df['y_cell_nearest'] = locations_df['name'].apply(lambda name: globals().get(f"{name}_cell_y"))
dump(locations_df, 'campgrounds_coordinates_gust.pkl')

['campgrounds_coordinates_gust.pkl']

for precip

In [3]:
# Define the locations and their coordinates
locations = {
    "Bern": (2595842.15, 1201548.69),
    "Brienz": (2646563.43, 1177603.88),
    "Buochs": (2674514.65, 1203568.08),
    "Interlaken": (2632995.62, 1171399.67),
    "Lugano": (2713926.15, 1094747.56),
    "Salavaux": (2569175.20, 1195846.24),
    "Sempach": (2657011.60, 1219718.50),
    "Gordevio": (2700590.96, 1119701.60),
    "Thun": (2614554.68, 1178201.33),
    "Morges": (2527116.10, 1150785.39)
}

# Open your datasets
path = '/scratch2/cwenger/ICON-CH1_monthly/ICON-CH1_08_2024_agg.nc'
ds = xr.open_dataset(path)  # Replace with your actual dataset

# Loop through each location to find the nearest x and y and save variables
for name, (loc_x, loc_y) in locations.items():
    # Find the nearest grid cell
    nearest_x = ds.sel(x=loc_x, y=loc_y, method="nearest").x.values
    nearest_y = ds.sel(x=loc_x, y=loc_y, method="nearest").y.values

    # Save variables dynamically
    globals()[f"{name}_cell_x"] = nearest_x
    globals()[f"{name}_cell_y"] = nearest_y


locations_df = pd.DataFrame([(name, coord[0], coord[1]) for name, coord in locations.items()], columns=['name', 'x_coordinate', 'y_coordinate'])
# Add x_cell and y_cell columns to the dataframe
locations_df['x_cell_nearest'] = locations_df['name'].apply(lambda name: globals().get(f"{name}_cell_x"))
locations_df['y_cell_nearest'] = locations_df['name'].apply(lambda name: globals().get(f"{name}_cell_y"))
dump(locations_df, 'campgrounds_coordinates_precip.pkl')

['campgrounds_coordinates_precip.pkl']

In [4]:
campgrounds_gust = load('campgrounds_coordinates_gust.pkl')
campgrounds_precip = load('campgrounds_coordinates_precip.pkl')

load wind and precip data

In [5]:
# Open your gust dataset
icon_path_gust = '/scratch2/cwenger/ICON-CH1_monthly/ICON-CH1_gust_data/ICON-CH1-EPS_2024-08-12T12_agg.nc'
ds_icon_gust = xr.open_dataset(icon_path_gust)

# Open your precipitation dataset
icon_path_precip = '/scratch2/cwenger/ICON-CH1_monthly/ICON-CH1_08_2024_agg.nc'
ds_icon_precip = xr.open_dataset(icon_path_precip)

select specific campground and extract window, for wind

In [6]:
# CHOOSE CAMPING SITE
c_name = 'Brienz'

# Define the window size (in grid points)
full_window = 35
half_window = int((full_window-1)/2)

center_x = campgrounds_gust[campgrounds_gust['name'].str.contains(c_name)]['x_cell_nearest'].values[0]
center_y = campgrounds_gust[campgrounds_gust['name'].str.contains(c_name)]['y_cell_nearest'].values[0]

# Get the indices of the nearest grid point
x_index = int(ds_icon_gust.x.to_index().get_loc(center_x))
y_index = int(ds_icon_gust.y.to_index().get_loc(center_y))

# Compute index bounds
x_start_idx = max(x_index - half_window, 0)
x_end_idx = min(x_index + half_window + 1, ds_icon_gust.sizes['x'])
y_start_idx = max(y_index - half_window, 0)
y_end_idx = min(y_index + half_window + 1, ds_icon_gust.sizes['y'])

# Extract the window for gust data
window_icon_gust = ds_icon_gust.isel(x=slice(x_start_idx, x_end_idx), y=slice(y_start_idx, y_end_idx))

select specific campground and extract window, for precip

In [7]:
# CHOOSE CAMPING SITE
c_name = 'Brienz'

# Define the window size (in grid points)
full_window = 35
half_window = int((full_window-1)/2)

center_x = campgrounds_precip[campgrounds_precip['name'].str.contains(c_name)]['x_cell_nearest'].values[0]
center_y = campgrounds_precip[campgrounds_precip['name'].str.contains(c_name)]['y_cell_nearest'].values[0]

# Get the indices of the nearest grid point
x_index = int(ds_icon_precip.x.to_index().get_loc(center_x))
y_index = int(ds_icon_precip.y.to_index().get_loc(center_y))

# Compute index bounds
x_start_idx = max(x_index - half_window, 0)
x_end_idx = min(x_index + half_window + 1, ds_icon_precip.sizes['x'])
y_start_idx = max(y_index - half_window, 0)
y_end_idx = min(y_index + half_window + 1, ds_icon_precip.sizes['y'])

# Extract the window for precipitation data
window_icon_precip = ds_icon_precip.isel(x=slice(x_start_idx, x_end_idx), y=slice(y_start_idx, y_end_idx))

select the lead time you're interested in

In [8]:
selected_date = '2024081212'
selected_leadtime = 0

window_icon_gust_time = window_icon_gust.sel(lead_time=selected_leadtime, forecast_reference_time='2024-08-12T12:00:00.000000000')
window_icon_precip_time = window_icon_precip.sel(date=selected_date, time=selected_leadtime)

now we want to calculate the probability that a certain threshold is exceeded. Let's start with precipitation: For Brienz, the threshold for warnlevel 2 is 13.6

In [9]:
precip_threshold = 13.6

# compute probability of exceedance across ensemble members
# first, we look at each grid point in the window and check whether the threshold is exceeded
exceedance = (window_icon_precip_time.tot_prec_1h > precip_threshold)
# we then count how many members at each grid point exceed the threshold
count_exceedance = exceedance.sum(dim='members')
# and then we divide by the total number of members to get the probability
probability_exceedance_precip = count_exceedance / window_icon_precip_time.sizes['members'] * 100

do the same for wind with 18 m/s

In [10]:
wind_threshold = 18

# compute probability of exceedance across ensemble members
# first, we look at each grid point in the window and check whether the threshold is exceeded
exceedance = (window_icon_gust_time.wind_speed_of_gust > wind_threshold)
# we then count how many members at each grid point exceed the threshold
count_exceedance = exceedance.sum(dim='realization')
# and then we divide by the total number of members to get the probability
probability_exceedance_gust = count_exceedance / window_icon_gust_time.sizes['realization'] * 100